In [95]:
import pandas as pd

In [96]:
import json

In [97]:
import sys

In [98]:
import os

In [99]:
os.environ['CUDA_VISIBLE_DEVICES'] = ""

In [100]:
import tensorflow as tf

In [101]:
import numpy as np

In [102]:
import os

In [103]:
from tqdm import tqdm

In [104]:
import cv2

In [105]:
temp = []

for class_name in ['r', 'p', 'b', 'bl', 'g']:
    df = pd.read_csv('/home/vladimir/workspace/data/kaggle_seals/results/comp4_det_test_patches0_{class_name}.txt'.format(class_name=class_name) ,
                   header=None, 
                   sep=' ')
    df.columns = ['file_index', 'confidence', 'x_min', 'y_min', 'x_max', 'y_max']
    
    df['class_name'] = class_name
    
    temp += [df]
    
    

In [106]:
for class_name in ['r', 'p', 'b', 'bl', 'g']:
    df = pd.read_csv('/home/vladimir/workspace/data/kaggle_seals/results/comp4_det_test_patches1_{class_name}.txt'.format(class_name=class_name) ,
                   header=None, 
                   sep=' ')
    df.columns = ['file_index', 'confidence', 'x_min', 'y_min', 'x_max', 'y_max']
    
    df['class_name'] = class_name
    
    temp += [df]

In [107]:
df = pd.concat(temp)

In [108]:
def rotate((x_min, y_min, x_max, y_max), angle=90, size=1000):
    if angle == 90:
        width = x_max - x_min
        return y_min, size - width - x_min, y_max, size - x_min
    elif angle == 180:
        return size - x_max, size - y_max, size - x_min, size - y_min
    elif angle == 270:
        width = x_max - x_min
        return size - y_max, x_min, size - y_min, x_min + width
    else:
        return (x_min, y_min, x_max, y_max)

In [109]:
df.head()

,file_index,confidence,x_min,y_min,x_max,y_max,class_name
0,18578_3900_1950.jpg,0.969,314.2,58.8,374.7,102.7,r
1,18578_3900_1950.jpg,0.961,409.5,312.9,455.6,347.7,r
2,18578_3900_1950.jpg,0.859,564.3,372.8,633.1,449.7,r
3,18578_3900_1950.jpg,0.612,359.8,427.1,417.7,462.2,r
4,18578_3900_1950.jpg,0.584,541.5,292.0,587.6,397.4,r


In [113]:
df.shape

(3174842, 7)

In [114]:
df = df[df['confidence'] > 0.6]

In [115]:
df.shape

(615263, 7)

In [123]:
df['file_name'] = df['file_index']

In [124]:
df['file_index'] = df['file_index'].str.replace('.jpg', '')

In [125]:
df['x_shift'] = df['file_index'].str.split('_').str.get(1).astype(int)
df['y_shift'] = df['file_index'].str.split('_').str.get(2).astype(int)

In [126]:
df['x_min'] += df['x_shift']
df['x_max'] += df['x_shift']

df['y_min'] += df['y_shift']
df['y_max'] += df['y_shift']


In [127]:
df.head()

,file_index,confidence,x_min,y_min,x_max,y_max,class_name,file_name,x_shift,y_shift
0,18578_3900_1950,0.969,4214.2,2008.8,4274.7,2052.7,r,18578_3900_1950.jpg,3900,1950
1,18578_3900_1950,0.961,4309.5,2262.9,4355.6,2297.7,r,18578_3900_1950.jpg,3900,1950
2,18578_3900_1950,0.859,4464.3,2322.8,4533.1,2399.7,r,18578_3900_1950.jpg,3900,1950
3,18578_3900_1950,0.612,4259.8,2377.1,4317.7,2412.2,r,18578_3900_1950.jpg,3900,1950
14,6906_975_1950,0.634,1494.1,2696.8,1562.3,2847.0,r,6906_975_1950.jpg,975,1950


In [128]:
df['test_id'] = df['file_index'].str.split('_').str.get(0)

In [129]:
df.head()

,file_index,confidence,x_min,y_min,x_max,y_max,class_name,file_name,x_shift,y_shift,test_id
0,18578_3900_1950,0.969,4214.2,2008.8,4274.7,2052.7,r,18578_3900_1950.jpg,3900,1950,18578
1,18578_3900_1950,0.961,4309.5,2262.9,4355.6,2297.7,r,18578_3900_1950.jpg,3900,1950,18578
2,18578_3900_1950,0.859,4464.3,2322.8,4533.1,2399.7,r,18578_3900_1950.jpg,3900,1950,18578
3,18578_3900_1950,0.612,4259.8,2377.1,4317.7,2412.2,r,18578_3900_1950.jpg,3900,1950,18578
14,6906_975_1950,0.634,1494.1,2696.8,1562.3,2847.0,r,6906_975_1950.jpg,975,1950,6906


In [130]:
df.shape

(615263, 11)

In [132]:
sample = pd.DataFrame(pd.read_csv('../data/sample_submission.csv')['test_id'])
sample['x'] = 1

In [133]:
sample.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18636 entries, 0 to 18635
Data columns (total 2 columns):
test_id    18636 non-null int64
x          18636 non-null int64
dtypes: int64(2)
memory usage: 291.3 KB


In [134]:
df['test_id'] = df['test_id'].astype(int)

In [135]:
df.head()

,file_index,confidence,x_min,y_min,x_max,y_max,class_name,file_name,x_shift,y_shift,test_id
0,18578_3900_1950,0.969,4214.2,2008.8,4274.7,2052.7,r,18578_3900_1950.jpg,3900,1950,18578
1,18578_3900_1950,0.961,4309.5,2262.9,4355.6,2297.7,r,18578_3900_1950.jpg,3900,1950,18578
2,18578_3900_1950,0.859,4464.3,2322.8,4533.1,2399.7,r,18578_3900_1950.jpg,3900,1950,18578
3,18578_3900_1950,0.612,4259.8,2377.1,4317.7,2412.2,r,18578_3900_1950.jpg,3900,1950,18578
14,6906_975_1950,0.634,1494.1,2696.8,1562.3,2847.0,r,6906_975_1950.jpg,975,1950,6906


In [137]:
df.head()

,file_index,confidence,x_min,y_min,x_max,y_max,class_name,file_name,x_shift,y_shift,test_id
0,18578_3900_1950,0.969,4214.2,2008.8,4274.7,2052.7,r,18578_3900_1950.jpg,3900,1950,18578
1,18578_3900_1950,0.961,4309.5,2262.9,4355.6,2297.7,r,18578_3900_1950.jpg,3900,1950,18578
2,18578_3900_1950,0.859,4464.3,2322.8,4533.1,2399.7,r,18578_3900_1950.jpg,3900,1950,18578
3,18578_3900_1950,0.612,4259.8,2377.1,4317.7,2412.2,r,18578_3900_1950.jpg,3900,1950,18578
14,6906_975_1950,0.634,1494.1,2696.8,1562.3,2847.0,r,6906_975_1950.jpg,975,1950,6906


In [138]:
class_name_dict = {'r': 'adult_males',
                  'b': 'adult_females',
                   'p': 'subadult_males',
                  'bl': 'juveniles',
                  'g': 'pups'}

In [139]:
g = pd.DataFrame(df.groupby(['test_id', 'class_name'])['x_min'].count()).reset_index()

In [140]:
g['class_name'] = g['class_name'].map(class_name_dict)

In [141]:
df1 = g.pivot(index='test_id', columns='class_name', values='x_min')

In [142]:
df1 = df1.reset_index()

In [143]:
df1.head()

class_name,test_id,adult_females,adult_males,juveniles,pups,subadult_males
0,0,24.0,9.0,2.0,NaN,3.0
1,1,2.0,2.0,NaN,NaN,NaN
2,2,NaN,1.0,NaN,NaN,NaN
3,3,1.0,NaN,2.0,NaN,2.0
4,4,1.0,6.0,NaN,NaN,1.0


In [144]:
df1.shape

(17288, 6)

In [145]:
sample = pd.DataFrame(pd.read_csv('../data/sample_submission.csv')['test_id'])
sample['x'] = 1

In [146]:
sample.head()

,test_id,x
0,0,1
1,1,1
2,2,1
3,3,1
4,4,1


In [147]:
sample = sample.merge(df1, on='test_id', how='left', left_index=True)

In [148]:
sample = sample.drop('x', 1).fillna(0)

In [149]:
for column in sample.columns:
    sample[column] = sample[column].astype(int)

In [150]:
sample.to_csv('../data/06.csv', index=False)

In [151]:
sample.head()

,test_id,adult_females,adult_males,juveniles,pups,subadult_males
0,0,24,9,2,0,3
1,1,2,2,0,0,0
2,2,0,1,0,0,0
3,3,1,0,2,0,2
4,4,1,6,0,0,1


In [152]:
sample.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 18636 entries, 0 to 17287
Data columns (total 6 columns):
test_id           18636 non-null int64
adult_females     18636 non-null int64
adult_males       18636 non-null int64
juveniles         18636 non-null int64
pups              18636 non-null int64
subadult_males    18636 non-null int64
dtypes: int64(6)
memory usage: 1019.2 KB


In [153]:
df.head()

,file_index,confidence,x_min,y_min,x_max,y_max,class_name,file_name,x_shift,y_shift,test_id
0,18578_3900_1950,0.969,4214.2,2008.8,4274.7,2052.7,r,18578_3900_1950.jpg,3900,1950,18578
1,18578_3900_1950,0.961,4309.5,2262.9,4355.6,2297.7,r,18578_3900_1950.jpg,3900,1950,18578
2,18578_3900_1950,0.859,4464.3,2322.8,4533.1,2399.7,r,18578_3900_1950.jpg,3900,1950,18578
3,18578_3900_1950,0.612,4259.8,2377.1,4317.7,2412.2,r,18578_3900_1950.jpg,3900,1950,18578
14,6906_975_1950,0.634,1494.1,2696.8,1562.3,2847.0,r,6906_975_1950.jpg,975,1950,6906


In [154]:
sample.describe()

,test_id,adult_females,adult_males,juveniles,pups,subadult_males
count,18636.000000,18636.000000,18636.000000,18636.000000,18636.000000,18636.000000
mean,9317.500000,17.584085,7.680189,2.688184,2.142037,2.920262
std,5379.894144,29.730962,10.068635,6.472352,7.376345,4.800576
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,4658.750000,1.000000,1.000000,0.000000,0.000000,0.000000
50%,9317.500000,5.000000,4.000000,1.000000,0.000000,1.000000
75%,13976.250000,22.000000,10.000000,3.000000,1.000000,4.000000
max,18635.000000,407.000000,137.000000,192.000000,122.000000,77.000000


In [155]:
df['id'] = df['test_id'].astype(str) + '_' + df['class_name']

In [69]:
g = df.groupby('id')

In [70]:
scores_input = tf.placeholder(tf.float32, shape=(None, ))
bboxes_input = tf.placeholder(tf.float32, shape=(None, 4))
bboxes_selection = tf.image.non_max_suppression(bboxes_input, scores_input, max_output_size=99, iou_threshold=0.2)

selected_bboxes = tf.gather(bboxes_input, bboxes_selection)

In [ ]:
# sess = tf.Session()
temp = []

for file_id, df_c in tqdm(g):
    sess = tf.Session()
    scores = df_c['confidence']
    bboxes = df_c[['x_min', 'y_min', 'x_max', 'y_max']].values
    
    result_bboxes = sess.run(selected_bboxes, feed_dict={scores_input: scores, bboxes_input: bboxes})
    selected_scores = sess.run(tf.gather(scores_input, bboxes_selection), feed_dict={scores_input: scores, bboxes_input: bboxes})
    
    df = pd.DataFrame(result_bboxes, columns = ['x_min', 'y_min', 'x_max', 'y_max'])
    df['id'] = file_id
    df['confidence'] = selected_scores
    
    temp += [df]
    
    sess.close()

  3%|▎         | 1448/57313 [00:51<1:06:50, 13.93it/s]

In [72]:
df = pd.concat(temp).reset_index(drop=True)

In [73]:
asd

NameError: name 'asd' is not defined

In [ ]:
df.shape

In [ ]:
df.head()

In [ ]:
# ind = (df['x_min'] < 2) | (df['y_min'] < 2) | (df['x_max'] > 1998) | (df['y_max'] > 1998)

In [ ]:
# df = df[~ind].reset_index(drop=True)

In [ ]:
sample_1 = pd.read_csv('../data/sampleSubmission.csv')

In [ ]:
sample_1['file_name'] = sample_1['id'].str.slice(0, 10) + '.jpg'

In [ ]:
sample_1.head()

In [ ]:
df.head()

In [ ]:
df['class_name'] = df['id'].str.get(11)

In [ ]:
df['file_name'] = df['id'].str.slice(0, 10) + '.jpg'

In [ ]:
keys = df['class_name'].unique()
class_map_dict = dict(zip(keys, range(9)))
class_map_dict_back = dict(zip(range(9), keys))

In [ ]:
class_map_dict

In [ ]:
df['class_name'] = df['class_name'].map(class_map_dict)

In [ ]:
df.head()

In [ ]:
g = df.groupby('file_name')

In [ ]:
scores_input = tf.placeholder(tf.float32, shape=(None, ))
bboxes_input = tf.placeholder(tf.float32, shape=(None, 4))
class_input = tf.placeholder(tf.float32, shape=(None, ))

bboxes_selection = tf.image.non_max_suppression(bboxes_input, scores_input, max_output_size=99, iou_threshold=0.5)

selected_bboxes = tf.gather(bboxes_input, bboxes_selection)

In [ ]:
sess = tf.Session()
temp = []

for file_name, df_c in tqdm(g):
    
    scores = df_c['confidence']
    bboxes = df_c[['x_min', 'y_min', 'x_max', 'y_max']].values
    class_name = df_c['class_name']
    
    
    result_bboxes = sess.run(selected_bboxes, feed_dict={scores_input: scores, bboxes_input: bboxes})    
    selected_scores = sess.run(tf.gather(scores_input, bboxes_selection), feed_dict={scores_input: scores, bboxes_input: bboxes})
    
    selected_class = sess.run(tf.gather(class_input, bboxes_selection), feed_dict={class_input: class_name, 
                                                                                   scores_input: scores,
                                                                                   bboxes_input: bboxes})
    
    df = pd.DataFrame(result_bboxes, columns = ['x_min', 'y_min', 'x_max', 'y_max'])
    df['file_name'] = file_name
    df['confidence'] = selected_scores
    df['class_name'] = selected_class
    
    temp += [df]
    
sess.close()

In [ ]:
df = pd.concat(temp).reset_index(drop=True)

In [ ]:
df.shape

In [ ]:
df['class_name'] = df['class_name'].astype(int).map(class_map_dict_back)

In [ ]:
df.head()

In [ ]:
# df_lif = pd.read_csv('../notebooks/lif_pred.csv')

In [ ]:
# df_lif.head()

In [ ]:
# df_lif['id'] = df_lif['file_name'].str.replace('.jpg', '') + '_' + df_lif['class_name']
# df_lif['confidence'] = 1

In [ ]:
# df = df[~df['file_name'].isin(df_lif['file_name'])]

In [ ]:
# df_lif = df_lif[df_lif['class_name'].notnull()]

In [ ]:
# df = pd.concat([df, df_lif]).reset_index(drop=True)

In [ ]:
df.shape

In [ ]:
colors = {'A': (0, 0, 255),
         'B': (255, 255, 255),
         'C': (255, 0, 0),
         'D': (127, 0, 255),
         'E': (255, 255, 0),
         'F': (255, 0, 127),
         'G': (0, 255, 255),
         'H': (0, 128, 255),
         'I': (51, 102, 0)}

In [ ]:
font = cv2.FONT_HERSHEY_SIMPLEX

In [ ]:
for file_name in tqdm(sample_1['file_name'].unique()):
    img = cv2.imread(os.path.join('/home/vladimir/workspace/data/dstl_cars/VOC2017/test', file_name))
    df_c = df[df['file_name'] == file_name]
    for i in df_c.index:
        class_name = df_c.loc[i, 'class_name']
        probs = df_c.loc[i, 'confidence']        
        boxes = df_c.loc[i, ['x_min', 'y_min', 'x_max', 'y_max']].astype(int).values        
#         for j, value in enumerate(boxes):            
        (x_min, y_min, x_max, y_max) = boxes
        img = cv2.rectangle(img, (x_min, y_min), (x_max, y_max), colors[class_name], 2)
        img = cv2.putText(img, str(int(100 * probs)), (x_max, y_min), font, 1, colors[class_name], 2)
    cv2.imwrite(os.path.join('test_preds', file_name), img)

In [ ]:
df.head()

In [ ]:
# df['x_min'] = df['x_min'].astype(int)
# df['y_min'] = df['y_min'].astype(int)
# df['x_max'] = df['x_max'].astype(int)
# df['y_max'] = df['y_max'].astype(int)

In [ ]:
sample = pd.read_csv('../data/sampleSubmission.csv')

In [ ]:
def helper(x):
    a = x['filtered_boxes']    
    result = []    
    for b in a:        
        result += ['{x}:{y}'.format(x=int(np.mean([b[0], b[2]])), 
                                      y=int(np.mean([b[1], b[3]])))]
    return '|'.join(result)

In [ ]:
df['x_mean'] = ((df['x_min'] + df['x_max']) / 2).astype(int)
df['y_mean'] = ((df['y_min'] + df['y_max']) / 2).astype(int)

In [ ]:
df['class_name'] = df['class_name'].str.upper()

In [ ]:
df.head()

In [ ]:
df['id'] = df['file_name'].str.replace('.jpg', '') + '_' + df['class_name']

In [ ]:
df.head()

In [ ]:
df['score'] = df['x_mean'].astype(str) + ':' + df['y_mean'].astype(str)

In [ ]:
# create file that will be changed to the Pascal Voc Format
result = []

g = df.groupby('file_name')

for file_name, df_c in tqdm(g):
    temp = []
    for i in df_c.index:
        x_min = int(df_c.loc[i, 'x_min'])
        y_min = int(df_c.loc[i, 'y_min'])
        x_max = int(df_c.loc[i, 'x_max'])
        y_max = int(df_c.loc[i, 'y_max'])
        class_name = df_c.loc[i, 'class_name']
        
        width = x_max - x_min
        height = y_max - y_min
        
        temp += [{
                'x': x_min,
                'y': y_min,
                'height': height,
                'width': width,
                'class': class_name
            }]
        result += [{'filename': '/home/vladimir/workspace/data/dstl_cars/VOC2017/test/' + file_name,
                   'annotations': temp,
                   'class': "image"}]

In [ ]:
result[0]

In [ ]:
json.dump(result, open('preds.json', 'w'), indent=4)

In [ ]:
g = pd.DataFrame(df.groupby('id')['score'].agg(lambda x: '|'.join(x))).reset_index()

In [ ]:
s = sample.merge(g, on='id', how='left')

In [ ]:
s = s[['id', 'score']]
s.columns = ['id', 'detections']

In [ ]:
s['detections'] = s['detections'].fillna('None')

In [ ]:
s.shape

In [ ]:
s.to_csv('../data/try_1000_vgg_32_06.csv', index=False)